## !CLEAR OUTPUT BEFORE COMMIT/PUSH!

### Imports

In [ ]:
from arcgis.gis import GIS
from arcgis.features import FeatureLayer, FeatureSet
from arcgis.mapping import WebMap, export_map
from arcgis.raster import ImageryLayer
from IPython import display

### Login/Locate Data

In [ ]:
gis = GIS(profile='adoezema_AGO')

In [ ]:
item_id = 'df1e1e2ce742480288318f22e4be721b'
feat_lyr = FeatureLayer.fromitem(gis.content.get(item_id))

### Helper Functions

In [ ]:
def get_meter_by_id(miu_id, lyr):
    keep_fields = ['Meter_Serial', 'Status', 'Meter_Installation', 'Meter_Type', 'Meter_Size', 'Manufacturer', 'Asset_ID', 'Billing_Account_Num', 'Consumption', 'Meter_Cycle', 'Meter_Route', 'Walk', 'Fund_ID', 'Meter_Number', 'Account_Name', 'Account_Number', 'Address', 'MIU_ID', 'Meter_Number1', 'Meter_Size1', 'Reading', 'Last_Reading', 'Meter_Read', 'Days_Since_Last_Read', 'Meter_Status', 'Route_Cycle', 'c2_times_Avg_Consumption', 'Days_of_Above_Avg_Cons', 'Occupancy_Status', 'Maintenance_Requested', 'Included_in_Analysis']
    #lat_long_sr = 4326
    #state_plane_sr = 3857
    wgs_84_sr = 3857
    return lyr.query(where=f"MIU_ID = {miu_id}", out_fields=keep_fields, out_sr=wgs_84_sr)

In [ ]:
def view_meter(query_result):
    m = gis.map()
    imagery = ImageryLayer('https://gisservices.oakgov.com/arcgis/rest/services/ImageServices/EnterpriseOrthoCurrentImageService/ImageServer')
    m.add_layer(imagery)
    m.add_layer(data)
    ext = {'spatialReference': {'latestWkid': 3857, 'wkid': 102100},
        'xmin': query_result.features[0].geometry['x'] - 50,
        'xmax': query_result.features[0].geometry['x'] + 50,
        'ymin': query_result.features[0].geometry['y'] - 50,
        'ymax': query_result.features[0].geometry['y'] + 50
        }
    m.extent=ext
    m.zoom=20
    return m

### Perform Query

In [ ]:
data = get_meter_by_id(1551193618, feat_lyr)

### View Result

In [ ]:
meter = view_meter(data)

In [ ]:
data_dict = [feat.as_dict for feat in data.features]

In [ ]:
%matplotlib agg
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
from cartopy.io.img_tiles import GoogleTiles
from io import BytesIO

In [ ]:
def create_map(feature):
    fig = plt.figure(figsize=(5,5))
    print(fig)
    url = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Street_Map/MapServer/tile/{z}/{y}/{x}.jpg'
    image = GoogleTiles(url=url)
    print(image)
    ax = fig.add_subplot(1,1,1, projection=image.crs)
    print(ax)
    ax.add_image(image, 15)
    xmin = feature["geometry"]["x"] - 500
    xmax = feature["geometry"]["x"] + 500
    ymin = feature["geometry"]["y"] - 500
    ymax = feature["geometry"]["y"] + 500
    print(xmax, xmin, ymax, ymin)
    ax.scatter(feature["geometry"]["x"], feature["geometry"]["y"],
         color='green', linewidth=2, marker='o'
         )
    print(ax.scatter) 
    io_bytes = BytesIO()
    print(io_bytes)
    plt.savefig(io_bytes,  format='png', bbox_inches='tight')
    print(plt)
    io_bytes.seek(0)
    return base64.b64encode(io_bytes.read()).decode()

In [ ]:
data_dict[0]["map"] = create_map(data_dict[0])

In [ ]:
from jinja2 import Environment, FileSystemLoader
env = Environment(loader=FileSystemLoader(r'C:\Users\adoezema\PycharmProjects\wtr_meter_report\templates'))
template = env.get_template("report_template.html")
template_variables = {
    'title': data.features[0].attributes['MIU_ID'],
    'features': data_dict
}
generate_html = template.render(template_variables)

In [ ]:
from weasyprint import HTML
HTML(string=generate_html).write_pdf(r"C:\Users\adoezema\PycharmProjects\wtr_meter_report\reports\report.pdf")